## Création de la base test

Populate three collections, 'mentions', 'events', and 'medias', of a 'nosqlproject' database:

Merge 'medias' to 'mentions' and keep fields of interest --> 'mentions_short':

Keep fields of interest --> 'events_short':

Create indexes to speed up requests:

## Prise en main

In [1]:
import pymongo
import pprint

In [2]:
client = pymongo.MongoClient("localhost", 27017)
#client.database_names()
client.nosqlproject.collection_names()

['mentions', 'events_short', 'medias', 'mentions_short', 'events']

In [3]:
client.nosqlproject.events_short.find_one()

{'Actor1Code': 'JUDJUD',
 'Actor1CountryCode': '',
 'Actor1Geo_CountryCode': 'US',
 'Actor1Name': 'DISTRICT COURT',
 'Day': 20160102,
 'GlobalEventID': 613300833,
 '_id': ObjectId('5a5db49342f86f1ffa1fc65d')}

In [4]:
client.nosqlproject.mentions_short.find_one()

{'FIPSCountryCode': 'UK',
 'GlobalEventID': 498730437,
 'MentionDocTone': 2.15686274509804,
 'MentionTimeDate': 20170101073000,
 'MentionType': 1,
 '_id': ObjectId('5a5db44f42f86f1ffa1fbe88')}

## Opinion moyenne dans le monde

In [5]:
DateStart = 20170101073000
DateEnd = 20170101073000

pipeline = [{'$lookup':
             {'from': "events_short",
              'let': {"event_id": "$GlobalEventID", "mention_type": "$MentionType", "mention_date": "$MentionTimeDate"},
              'pipeline': [
                  {'$match':
                   {'$expr':
                    {'$and':
                     [
                         {'$lte': ["$$mention_date",  DateStart]},
                         {'$gte': ["$$mention_date",  DateEnd]},
                         {'$eq': ["$$mention_type",  1]},  # 1 = WEB
                         # join on GlobalEventID
                         {'$eq': ["$GlobalEventID",  "$$event_id"]},
                         #{'$or': [{'$eq': ["$Actor1Code",  "USAGOV"]}, {'$and': [{'$eq': ["$Actor1Code",  "GOV"]}, {'$eq': ["$Actor1Geo_CountryCode",  "US"]}]}]}
                         {'$eq': ["$Actor1Code",  "USAGOV"]}
                     ]
                     }
                    }
                   }
              ],
              'as': "mentions_usa"
              }
             },
            {'$match': {"mentions_usa": {'$ne': []}}
             },
            {'$group': {"_id": "$FIPSCountryCode", "tone": {
                '$avg': "$MentionDocTone"}, "count": {'$sum': 1}}}
            ]

list(client.nosqlproject.mentions_short.aggregate(pipeline))

[{'_id': None, 'count': 1, 'tone': -6.9277108433735},
 {'_id': 'IS', 'count': 2, 'tone': -2.89330922242315},
 {'_id': 'UK', 'count': 1, 'tone': -6.57596371882086},
 {'_id': 'US', 'count': 4, 'tone': -5.13698630136987}]

## Évolution de la perception d'un pays

In [6]:
DateStart = 20170101073000
DateEnd = 20170101073000
CountryCode = 'US'

pipeline = [{'$lookup':
             {'from': "events_short",
              'let': {"event_id": "$GlobalEventID", "mention_type": "$MentionType", "mention_date": "$MentionTimeDate", "country_code": "$FIPSCountryCode"},
              'pipeline': [
                  {'$match':
                   {'$expr':
                    {'$and':
                     [
                         {'$lte': ["$$mention_date",  DateStart]},
                         {'$gte': ["$$mention_date",  DateEnd]},
                         {'$eq': ["$$mention_type",  1]},  # 1 = WEB
                         # join on GlobalEventID
                         {'$eq': ["$GlobalEventID",  "$$event_id"]},
                         #{'$or': [{'$eq': ["$Actor1Code",  "USAGOV"]}, {'$and': [{'$eq': ["$Actor1Code",  "GOV"]}, {'$eq': ["$Actor1Geo_CountryCode",  "US"]}]}]},
                         {'$eq': ["$Actor1Code",  "USAGOV"]},
                         {'$eq': ["$$country_code",  CountryCode]}
                     ]
                     }
                    }
                   }
              ],
              'as': "mentions_usa"
              }
             },
            {'$match': {"mentions_usa": {'$ne': []}}
             }
            ]

list(client.nosqlproject.mentions_short.aggregate(pipeline))

[{'FIPSCountryCode': 'US',
  'GlobalEventID': 613301532,
  'MentionDocTone': -5.13698630136987,
  'MentionTimeDate': 20170101073000,
  'MentionType': 1,
  '_id': ObjectId('5a5db45042f86f1ffa1fc5f6'),
  'mentions_usa': [{'Actor1Code': 'USAGOV',
    'Actor1CountryCode': 'USA',
    'Actor1Geo_CountryCode': 'US',
    'Actor1Name': 'OBAMA',
    'Day': 20170101,
    'GlobalEventID': 613301532,
    '_id': ObjectId('5a5db49442f86f1ffa1fc919')}]},
 {'FIPSCountryCode': 'US',
  'GlobalEventID': 613301533,
  'MentionDocTone': -5.13698630136987,
  'MentionTimeDate': 20170101073000,
  'MentionType': 1,
  '_id': ObjectId('5a5db45042f86f1ffa1fc5f7'),
  'mentions_usa': [{'Actor1Code': 'USAGOV',
    'Actor1CountryCode': 'USA',
    'Actor1Geo_CountryCode': 'IS',
    'Actor1Name': 'OBAMA',
    'Day': 20170101,
    'GlobalEventID': 613301533,
    '_id': ObjectId('5a5db49442f86f1ffa1fc91a')}]},
 {'FIPSCountryCode': 'US',
  'GlobalEventID': 613301534,
  'MentionDocTone': -5.13698630136987,
  'MentionTimeDate